# Конечные автоматы

Определение и интуитивные представления см. в [википедии](https://ru.wikipedia.org/wiki/%D0%94%D0%B5%D1%82%D0%B5%D1%80%D0%BC%D0%B8%D0%BD%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BA%D0%BE%D0%BD%D0%B5%D1%87%D0%BD%D1%8B%D0%B9_%D0%B0%D0%B2%D1%82%D0%BE%D0%BC%D0%B0%D1%82) и в книге:
* Д. Хопкрофт, Р. Мотвани, and Д. Ульман, Введение в теорию автоматов, языков и вычислений.


# Задание

Разбейтесь группами по три-четыре человека и реализуйте следующие функции:
* Преобразование NFA -> EpsilonNFA и функция in(::Any, ::EpsilonNFA)
* Преобразование EpsilonNFA -> DFA
* Преобразование RE -> EpsilonNFA
* Преобразование DFA -> RE (повышенная сложность)


# Примеры вопросов на зачёте
* Как работает алгоритм преобразования RE -> EpsilonNFA?
* Придумайте конечный автомат, допускающий двоичные слова, в которых есть подстрока `11`.
* Опишите язык, слова которого допускает регулярное выражение `1(0|1)*`.
* Упростите регулярное выражение `(0|1)*(0|1|1)*11`
* Пересечение регулярных языков является регулярным языком? Почему?

In [1]:
"""
Детерминированный конечный автомат
Deterministic Finite Automaton
"""
struct DFA
    "Функция перехода. Текущему состоянию и текущему входному символу сопоставляется следующее состояние"
    transition::Dict{Int, Dict{Char, Int}}
    "Начальное состояние"
    start::Int
    "Множество допускающих состояний"
    accept::Set{Int}
end


"""
Содержит ли язык данное слово?
"""
function Base.in(word, dfa::DFA)::Bool
    state = dfa.start
    for letter in word
        state = dfa.transition[state][letter]
    end
    
    return state in dfa.accept
end

Base.in

In [2]:
# Пример конечного автомата, допускающего все слова, содержащие "01"

dfa = DFA(Dict(1 => Dict('0' => 2, '1' => 1), 2 => Dict('0' => 2, '1' => 3), 3 => Dict('0' => 3, '1' => 3)), 1, Set([3]))

"0000010" ∈ dfa, "10000" ∈ dfa

(true, false)

In [3]:
# Пример конечного автомата, допускающего все слова с нечётным числом нулей

dfa = DFA(Dict(1 => Dict('0' => 2, '1' => 1), 2 => Dict('0' => 1, '1' => 2)), 1, Set([2]))

"111111" ∈ dfa, "0101101011001110111" ∈ dfa

(false, true)

In [4]:
"""
Недетерминированный конечный автомат. Поля аналогичны DFA, кроме функции перехода.
"""
struct NFA
    "Функция перехода. Текущему состоянию и текущему входному символу сопоставляется множество следующих состояний"
    transition::Dict{Int, Dict{Char, Set{Int}}}
    "Начальное состояние"
    start::Int
    "Множество допускающих состояний"
    accept::Set{Int}
end


"""
Содержит ли язык данное слово?
"""
function Base.in(word, nfa::NFA)::Bool
    states = Set{Int}()
    push!(states, nfa.start)
    for symbol in word
        states = union([get(nfa.transition[state], symbol, Set{Int}()) for state in states]...)::Set{Int}
    end
    
    return ~isempty(intersect(states, nfa.accept))
end

Base.in

In [5]:
# Пример конечного автомата, допускающего все слова, окончавающиеся на 1

nfa = NFA(Dict{Int, Dict{Char, Set{Int}}}(1 => Dict('0' => Set(1), '1' => Set([1, 2])), 2 => Dict()), 1, Set([2]))
"000010" ∈ nfa, "111001001" ∈ nfa

(false, true)

In [6]:
"""
Преобразование DFA -> NFA
"""
function NFA(dfa::DFA)::NFA
    trans = Dict(from => Dict(sym => Set(to) for (sym, to) in pairs(perstate))
                 for (from, perstate) in pairs(dfa.transition))

    return NFA(trans, dfa.start, dfa.accept)
end

NFA

In [7]:
"""
Недетерминированный конечный автомат с эпсилон-переходами.
Поля аналогичны NFA, кроме эпсилон-переходов, заданных в поле epsilon.
"""
struct EpsilonNFA
    "Функция перехода. Текущему состоянию и текущему входному символу сопоставляется множество следующих состояний"
    transition::Dict{Int, Dict{Char, Set{Int}}}
    "Эпсилон-переходы"
    epsilon::Dict{Int, Set{Int}}
    "Начальное состояние"
    start::Int
    "Множество допускающих состояний"
    accept::Set{Int}
end

EpsilonNFA

In [8]:
"""
Преобразование NFA -> EpsilonNFA
"""
function EpsilonNFA(nfa::NFA)::EpsilonNFA
    # ЗДЕСЬ КОД
end


"""
Содержит ли язык данное слово?
"""
function Base.in(word, enfa::EpsilonNFA)::Bool
    # ЗДЕСЬ КОД
end

Base.in

In [9]:
"""
Преобразование EpsilonNFA -> DFA
"""
function DFA(nfa::EpsilonNFA)::DFA
    # ЗДЕСЬ КОД
end

DFA

In [10]:
"""
Регулярные выражения
"""
abstract type RegularExpression end

const RE = RegularExpression  # Сокращение

"""
Пустое регулярное выражение
"""
struct EmptyRE <: RE
end


"""
Регулярное выражение из одной буквы алфавита
"""
struct LetterRE <: RE
    value::Char
end


"""
Регулярное выражение, полученное конкатенацией
"""
struct ConcatRE <: RE
    values::Vector{RE}
end


"""
Регулярные выражение, полученное объединением
"""
struct UnionRE <: RE
    values::Vector{RE}
end


"""
Регулярное выражение -- звезда Клини
"""
struct KleeneRE <: RE
    value::RE
end

;

In [11]:
# Пример регулярного выражения, описывающего язык из двоичных слов, оканчивающихся на 1
zeroOrOne = UnionRE(RE[LetterRE('0'), LetterRE('1')])
re = ConcatRE(RE[KleeneRE(zeroOrOne), LetterRE('1')])

# Текстом такое выражение обычно записывается как (0|1)*1

ConcatRE(RegularExpression[KleeneRE(UnionRE(RegularExpression[LetterRE('0'), LetterRE('1')])), LetterRE('1')])

In [12]:
"""
Преобразование RE -> EpsilonNFA
"""
function EpsilonNFA(re::RE)::EpsilonNFA
    # ЗДЕСЬ КОД
end

EpsilonNFA

In [13]:
"""
Преобразование DFA -> RE
"""
function RE(dfa::DFA)::RE
    # ЗДЕСЬ КОД
end

RE